# Creating a Network of Images

## Load database

In [268]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [269]:
import chromadb
from tqdm.notebook import tqdm
from pyvis.network import Network
from random import shuffle

In [270]:
client = chromadb.PersistentClient(path="ce_vector_db")
db = client.get_or_create_collection(name="congruence_engine")

In [271]:
db.count()

11863

In [272]:
from itertools import combinations

In [276]:
data = db.get()
ids = data['ids']
shuffle(ids)
ids = ids[:1000]
#dbid2netid = {idx: i for i, idx in enumerate(ids)}

In [277]:
threshold = .1

In [306]:
dbid2netid = {e: i+1 for i,e in enumerate(ids)}

net = Network(height="750px", 
              width="100%", 
              bgcolor="#222222", 
              font_color="white", 
              cdn_resources='remote', 
              #filter_menu=True, 
              #select_menu=True,
              notebook=True)

for idx in ids:
    el = db.get(ids=[idx], include=['embeddings','metadatas','documents'])

    net.add_node(dbid2netid[idx], shape='image', 
                     label=el['ids'][0], 
                     image = el['metadatas'][0]['img_url'],
                     title = el['documents'][0])
    
for idx in ids: 
    el = db.get(ids=[idx], include=['embeddings','metadatas','documents'])
    nn = db.query(
    query_embeddings = el['embeddings'],
             n_results=100
             )

    for i,neighbour in enumerate(nn['ids'][0]):
        if idx == neighbour: continue
        if nn['distances'][0][i] < threshold:
            if idx in dbid2netid and neighbour in dbid2netid:
                net.add_edge(dbid2netid[idx], 
                             dbid2netid[neighbour], 
                             weight=nn['distances'][0][i])# label=f'{s}_{t}'

len(net.nodes),len(net.edges)

(1000, 323)

In [307]:
type(net.nodes)

list

In [308]:
edge_ids = []
for e in net.edges:
    if e['from'] != e['to']:
        edge_ids.extend([e['from'],e['to']])
edge_ids = set(edge_ids)

In [309]:

print(len(net.nodes))
for n in net.nodes:
    if n['id'] not in edge_ids:
        net.nodes.remove(n)
print(len(net.nodes))

1000
570


In [310]:
net.show("object_network.html",) # show_buttons(filter_=['physics'])

object_network.html


# Fin.